# فصل ۷: ساخت برنامه‌های چت
## شروع سریع API مدل‌های گیت‌هاب

این دفترچه از [مخزن نمونه‌های Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) گرفته شده است که شامل دفترچه‌هایی برای دسترسی به سرویس‌های [Azure OpenAI](notebook-azure-openai.ipynb) می‌باشد.


# مروری کلی  
«مدل‌های زبانی بزرگ توابعی هستند که متن را به متن نگاشت می‌کنند. وقتی یک رشته متنی به عنوان ورودی داده می‌شود، مدل زبانی بزرگ سعی می‌کند متنی را که قرار است بعد از آن بیاید پیش‌بینی کند»(1). این دفترچه «شروع سریع» کاربران را با مفاهیم سطح بالای LLM، نیازمندی‌های اصلی بسته برای شروع کار با AML، مقدمه‌ای ساده بر طراحی پرامپت و چند مثال کوتاه از کاربردهای مختلف آشنا می‌کند.


## فهرست مطالب  

[مروری کلی](../../../../07-building-chat-applications/python)  
[نحوه استفاده از سرویس OpenAI](../../../../07-building-chat-applications/python)  
[۱. ساخت سرویس OpenAI خودتان](../../../../07-building-chat-applications/python)  
[۲. نصب](../../../../07-building-chat-applications/python)    
[۳. اطلاعات ورود](../../../../07-building-chat-applications/python)  

[موارد استفاده](../../../../07-building-chat-applications/python)    
[۱. خلاصه‌کردن متن](../../../../07-building-chat-applications/python)  
[۲. دسته‌بندی متن](../../../../07-building-chat-applications/python)  
[۳. تولید نام‌های جدید برای محصول](../../../../07-building-chat-applications/python)  
[۴. آموزش دقیق‌تر یک دسته‌بند](../../../../07-building-chat-applications/python)  

[منابع](../../../../07-building-chat-applications/python)


### اولین پرامپت خود را بسازید  
این تمرین کوتاه، مقدمه‌ای ساده برای ارسال پرامپت به یک مدل در Github Models برای یک کار ساده مثل "خلاصه‌سازی" ارائه می‌دهد.

**مراحل**:  
۱. اگر هنوز انجام نداده‌اید، کتابخانه `azure-ai-inference` را در محیط پایتون خود نصب کنید.  
۲. کتابخانه‌های کمکی استاندارد را بارگذاری کرده و اعتبارنامه Github Models را تنظیم کنید.  
۳. یک مدل مناسب برای کار خود انتخاب کنید  
۴. یک پرامپت ساده برای مدل بسازید  
۵. درخواست خود را به API مدل ارسال کنید!


### ۱. نصب `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

### ۲. وارد کردن کتابخانه‌های کمکی و ایجاد اعتبارنامه‌ها


In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### ۳. پیدا کردن مدل مناسب  
مدل‌های GPT-3.5-turbo یا GPT-4 می‌توانند زبان طبیعی را درک کرده و تولید کنند.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## ۴. طراحی پرامپت  

«جادوی مدل‌های زبانی بزرگ این است که با آموزش دیدن برای کمینه کردن خطای پیش‌بینی روی حجم عظیمی از متن، این مدل‌ها در نهایت مفاهیمی را یاد می‌گیرند که برای این پیش‌بینی‌ها مفید هستند. برای مثال، آن‌ها مفاهیمی مانند»(۱):

* چگونه املای کلمات را بنویسند
* قواعد گرامری چگونه کار می‌کند
* چطور پارافرایز کنند
* چطور به سوالات پاسخ دهند
* چطور یک مکالمه را پیش ببرند
* چطور به زبان‌های مختلف بنویسند
* چطور کدنویسی کنند
* و غیره

#### چگونه یک مدل زبانی بزرگ را کنترل کنیم  
«در میان تمام ورودی‌های یک مدل زبانی بزرگ، بدون شک تاثیرگذارترین آن‌ها، متن پرامپت است»(۱).

مدل‌های زبانی بزرگ را می‌توان به چند روش برای تولید خروجی هدایت کرد:

دستورالعمل: به مدل بگویید چه می‌خواهید  
تکمیل: مدل را وادار کنید تا ابتدای چیزی که می‌خواهید را کامل کند  
نمایش: به مدل نشان دهید چه می‌خواهید، با یکی از این دو روش:  
چند مثال در پرامپت  
صدها یا هزاران مثال در یک دیتاست آموزش فاین‌تیون شده»


#### سه راهنمای اصلی برای ساخت پرامپت وجود دارد:

**نشان بدهید و توضیح دهید.** به وضوح مشخص کنید چه می‌خواهید؛ چه با دستورالعمل، چه با مثال، یا ترکیبی از هر دو. اگر می‌خواهید مدل یک لیست را به ترتیب حروف الفبا مرتب کند یا یک پاراگراف را بر اساس احساس دسته‌بندی کند، به آن نشان دهید که دقیقا همین را می‌خواهید.

**داده باکیفیت ارائه دهید.** اگر می‌خواهید یک دسته‌بند بسازید یا مدل را وادار کنید از یک الگو پیروی کند، مطمئن شوید که مثال‌های کافی دارید. حتما مثال‌های خود را بازبینی کنید — مدل معمولا آنقدر باهوش هست که اشتباهات ساده املایی را تشخیص دهد و به شما پاسخ دهد، اما ممکن است فرض کند این اشتباهات عمدی هستند و این می‌تواند روی پاسخ تاثیر بگذارد.

**تنظیمات خود را بررسی کنید.** تنظیمات temperature و top_p تعیین می‌کنند که مدل تا چه حد در تولید پاسخ قطعی عمل کند. اگر از مدل پاسخی می‌خواهید که فقط یک جواب درست دارد، بهتر است این مقادیر را پایین بگذارید. اگر به دنبال پاسخ‌های متنوع‌تر هستید، می‌توانید آن‌ها را بالاتر تنظیم کنید. رایج‌ترین اشتباه افراد با این تنظیمات این است که فکر می‌کنند این‌ها کنترل‌کننده «باهوشی» یا «خلاقیت» مدل هستند.


منبع: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## خلاصه کردن متن  
#### چالش  
متن را با اضافه کردن «tl;dr:» در انتهای آن خلاصه کنید. توجه کنید که مدل چطور بدون هیچ دستور اضافه‌ای می‌تواند چندین کار مختلف را انجام دهد. می‌توانید با استفاده از درخواست‌های توصیفی‌تر به جای tl;dr، رفتار مدل را تغییر دهید و خلاصه‌ای که دریافت می‌کنید را شخصی‌سازی کنید(3).

تحقیقات اخیر نشان داده‌اند که پیش‌آموزش روی حجم بزرگی از متن و سپس تنظیم دقیق روی یک وظیفه خاص، پیشرفت چشمگیری در بسیاری از وظایف و معیارهای NLP ایجاد کرده است. اگرچه معماری این روش معمولاً مستقل از وظیفه است، اما همچنان به داده‌های تنظیم دقیق مخصوص هر وظیفه با هزاران یا ده‌ها هزار نمونه نیاز دارد. در مقابل، انسان‌ها معمولاً می‌توانند یک وظیفه زبانی جدید را تنها با چند مثال یا دستور ساده انجام دهند؛ چیزی که سیستم‌های NLP فعلی هنوز تا حد زیادی با آن مشکل دارند. در اینجا نشان می‌دهیم که بزرگ‌تر کردن مدل‌های زبانی، عملکرد مستقل از وظیفه و چندنمونه‌ای را به طور قابل توجهی بهبود می‌بخشد و گاهی حتی با روش‌های تنظیم دقیق پیشرفته قبلی رقابت می‌کند.



Tl;dr


# تمرین‌ها برای چندین مورد استفاده  
1. خلاصه کردن متن  
2. دسته‌بندی متن  
3. تولید نام‌های جدید برای محصولات


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## دسته‌بندی متن  
#### چالش  
موارد را بر اساس دسته‌هایی که هنگام استنتاج ارائه می‌شوند، دسته‌بندی کنید. در مثال زیر، هم دسته‌ها و هم متنی که باید دسته‌بندی شود را در پرامپت قرار داده‌ایم (*playground_reference).

سؤال مشتری: سلام، یکی از کلیدهای کیبورد لپ‌تاپ من اخیراً خراب شده و نیاز به تعویض دارم:

دسته‌بندی شده:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## تولید نام‌های جدید برای محصولات
#### چالش
نام‌هایی برای محصولات با استفاده از کلمات نمونه بسازید. در اینجا، اطلاعاتی درباره محصولی که قرار است برای آن نام تولید کنیم، در متن قرار داده‌ایم. همچنین یک مثال مشابه ارائه شده تا الگوی مورد انتظار را نشان دهیم. مقدار دما (temperature) را نیز بالا تنظیم کرده‌ایم تا پاسخ‌ها خلاقانه‌تر و متنوع‌تر باشند.

توضیح محصول: دستگاه تهیه میلک‌شیک خانگی  
کلمات کلیدی: سریع، سالم، جمع‌وجور  
نام‌های محصول: هوم‌شیکر، فیت‌شیکر، کوئیک‌شیک، شیک‌میکر

توضیح محصول: یک جفت کفش که برای هر سایز پایی مناسب است.  
کلمات کلیدی: قابل‌انطباق، فیت، اومنی‌فیت.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# منابع  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [نمونه‌های OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [بهترین روش‌ها برای تنظیم دقیق GPT-3 جهت طبقه‌بندی متن](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# برای کمک بیشتر  
[تیم تجاری‌سازی OpenAI](AzureOpenAITeam@microsoft.com)


# مشارکت‌کنندگان
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**سلب مسئولیت**:
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطا یا نادقتی باشند. نسخه اصلی سند به زبان مادری آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم یا تفسیر نادرست ناشی از استفاده از این ترجمه نداریم.
